## 1. Знакомимся с данными

--Укажите название города с максимальным весом единичной доставки.

select 
    city_name,
    max(weight)
from sql.city c
    left join sql.shipment s on c.city_id = s.city_id
where weight is not null
group by 1
order by 2 desc

-- Сколько различных производителей грузовиков перечислено в таблице truck?

select 
count(distinct make)
from sql.truck t
--where weight is not null
--group by 1
--order by 2 desc

--→ Как зовут водителя (first_name), который совершил наибольшее количество доставок одному клиенту?

select
    d.first_name,
    count(c.cust_id)
from sql.driver d
    join sql.shipment s on d.driver_id = s.driver_id
    join sql.customer c on s.cust_id = c.cust_id
group by 1
order by 2 desc

--Укажите даты первой и последней по времени доставок в таблице shipment. Ответ введите в формате ДД.ММ.ГГГГ.

select 
    min(ship_date),
    max(ship_date)
from sql.shipment s
--order by 1 desc

--Укажите имя клиента, получившего наибольшее количество доставок за 2017 год.

select 
    c.cust_name,
    count(s.ship_id)
from sql.customer c
    join sql.shipment s on c.cust_id = s.cust_id
where s.ship_date between to_date('01.01.2017', 'dd.mm.yyyy') and to_date('31.12.2017', 'dd.mm.yyyy')
group by 1
order by 2 desc

## 2. UNION

ВИДЫ UNION

Оператор присоединения существует в двух вариантах:

UNION выводит только уникальные записи;
UNION ALL присоединяет все строки последующих таблиц к предыдущим, без ограничений по уникальности.

Важно! UNION оставляет только уникальные значения, а потому требует дополнительных вычислительных мощностей и памяти (в данном случае можно провести аналогию с DISTINCT). Поэтому если вы уверены в отсутствии дубликатов в данных или они вам не важны, предпочтительнее использовать UNION ALL.

UNION ALL — присоединяет любой результат запроса к другому «снизу» при условии, что у них одинаковая структура, а именно:
одинаковый тип данных;
одинаковое количество столбцов;
одинаковый порядок столбцов согласно типу данных.


СИНТАКСИС

SELECT
         n columns
FROM 
         table_1
UNION ALL
SELECT 
         n columns
FROM 
         table_2
...
UNION ALL
SELECT 
         n columns
FROM 
         table_n

                                                                Пример:

SELECT
         c.city_id object_name, 'id города' object_type
FROM 
         sql.city c
UNION ALL
SELECT
         d.driver_id other_name, 'id водителя' other_type
FROM 
         sql.driver d
UNION ALL
SELECT
         s.ship_id, 'id доставки'
FROM 
         sql.shipment s
UNION ALL
SELECT
         c.cust_id, 'id клиента'
FROM 
         sql.customer c
UNION ALl
SELECT
         t.truck_id, 'id грузовика'
FROM 
         sql.truck t
ORDER BY 1

Сортировка каждого запроса:

(SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
ORDER BY 1
LIMIT 1)
UNION ALL
(SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1)

Задание 2.1
Напишите запрос, который создаёт уникальный алфавитный справочник всех городов, штатов, имён водителей и производителей грузовиков.
Результатом запроса должны быть два столбца: название и тип объекта (city, state, driver, truck). Отсортируйте список по названию объекта, а затем — по типу.

SELECT city_name as object_name, 'city' as object_type
    FROM sql.city
UNION 
SELECT state, 'state' 
    FROM sql.city
UNION 
SELECT first_name, 'driver' 
    FROM sql.driver
UNION 
SELECT make, 'truck' 
    FROM sql.truck
ORDER BY 1, 2

Задание 2.2
Напишите запрос, который соберёт имена всех упомянутых городов и штатов с таблицы city. Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

SELECT city_name as object_name
    FROM sql.city
UNION all
SELECT state
    FROM sql.city
order by object_name

Задание 2.3
Выполнив предыдущий запрос, мы получили города с одинаковыми названиями, но находящиеся в разных штатах, а также большое количество дублирующихся названий штатов.
Перепишите предыдущий запрос так, чтобы остались только уникальные названия городов и штатов. Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

SELECT  distinct city_name as object_name
    FROM sql.city
UNION
SELECT state
    FROM sql.city
order by object_name

## 3. UNION и ограничение типов данных

Любой тип данных может быть приведён к текстовому формату — эту возможность целесообразно использовать для соединения разнородных сущностей. Главное — помнить, что сортировка текста отличается от сортировки чисел и дат.
column_name::column_type

SELECT 
         c.city_id::text
FROM
         sql.city c
UNION ALL
SELECT 
         cc.city_name
FROM
         sql.city cc

Задание 3.1
Напишите запрос, который объединит в себе все почтовые индексы водителей и их телефоны в единый столбец-справочник. Также добавьте столбец с именем водителя и столбец с типом контакта (phone или zip в зависимости от типа).
Отсортируйте список по столбцу с контактными данными в порядке возрастания, а затем — по имени водителя.

select d.zip_code::text index_code, d.first_name driver_name, 'zip' contact_type
    from sql.driver d
union all
select d.phone, d.first_name, 'phone'
    from sql.driver d
order by 1, 2

## 4. UNION ALL и промежуточные итоги

SELECT
         c.city_name,
         c.population
FROM
         sql.city c
UNION ALL
SELECT
         'total',
         SUM(c.population)
FROM
         sql.city c
ORDER BY 2 DESC

Blank Advanced Problem
Напишите запрос, который выводит общее число доставок total_shipments, а также количество доставок в каждый день. Необходимые столбцы: date_period, cnt_shipping.
Не забывайте о единой типизации. Упорядочьте по убыванию столбца date_period.

SELECT s.ship_date::text date_period, COUNT(*) cnt_shipment
FROM sql.shipment s 
GROUP BY 1
UNION ALL
SELECT 'total_shipments', count(*)
FROM sql.shipment s
ORDER BY 1 desc

## 5. UNION и дополнительные условия

SELECT
         d.first_name,
         d.last_name,
         'телефон заполнен' phone_info
FROM
         sql.driver d
WHERE d.phone IS NOT NULL
UNION
SELECT
         d.first_name,
         d.last_name,
         'телефон не заполнен' phone_info
FROM
         sql.driver d
WHERE d.phone IS NULL

Задание 5.1
Напишите запрос, который выведет все города и штаты, в которых они расположены, а также информацию о том, была ли осуществлена доставка в этот город:
если в город была осуществлена доставка, то выводим 'доставка осуществлялась';
если нет — выводим 'доставка не осуществлялась'.
Столбцы к выводу: city_name, state, shipping_status.
Отсортируйте в алфавитном порядке по городу, а затем — по штату.

SELECT
        c.city_name,
        c.state,
        'доставка осуществлялась' shipping_status
FROM
         sql.city c
         left join sql.shipment s on c.city_id = s.city_id
WHERE s.ship_id IS NOT NULL

UNION

SELECT 
        c.city_name,
        c.state,
        'доставка не осуществлялась' shipping_status
FROM
         sql.city c
         left join sql.shipment s on c.city_id = s.city_id
WHERE s.ship_id IS NULL
order by 1,2

Задание 5.2
Напишите запрос, который выводит два столбца: city_name и shippings_fake. Выведите города, куда совершались доставки.
Пусть первый столбец содержит название города, а второй формируется так:
если в городе было более десяти доставок, вывести количество доставок в этот город как есть;
иначе — вывести количество доставок, увеличенное на пять.
Отсортируйте по убыванию получившегося «нечестного» количества доставок, а затем — по имени в алфавитном порядке.

SELECT
        c.city_name,
        count(s.ship_id) shippings_fake
FROM
         sql.city c
         join sql.shipment s on c.city_id = s.city_id
group by c.city_id
having count(s.ship_id)>10
UNION
SELECT 
        c.city_name,
        count(s.ship_id)+5 shippings_fake
FROM
         sql.city c
         join sql.shipment s on c.city_id = s.city_id
group by c.city_id
having count(s.ship_id)<=10
order by 2 desc, 1

## 6. UNION и ручная генерация

Задание 6.1. Напишите запрос, который выберет наибольшее из значений: 1000000; 541; -500; 100.

SELECT 
1000000 as number
UNION 
SELECT 
        541
UNION 
SELECT
         -500
UNION 
SELECT
         100   
order by 1 desc
limit 1

Задание 6.2. Мы помним, что сортировка для числовых и строковых типов данных отличается.
Построив запрос по аналогии с примером, приведите значения к текстовому типу данных, сравните и выберите наибольшее из них: 1000000; 541; -500; 100.

SELECT 
'1000000' as number
UNION 
SELECT 
        '541'
UNION 
SELECT
         '-500'
UNION 
SELECT
         '100'       
order by 1 desc
limit 1

Задание 6.3. Построив запрос по аналогии с примером, найдите самое большое значение из перечисленных операторов: + ; - ; = ; / .

SELECT 
    '+' as number
UNION 
SELECT 
        '-'
UNION 
SELECT
         '='
UNION 
SELECT
         '/'
order by 1 desc
limit 1

## 7. EXCEPT

ИСКЛЮЧАЕМ ПОВТОРЯЮЩИЕСЯ ДАННЫЕ

нужно узнать, в какие города осуществлялась доставка, за исключением тех, в которых проживают водители

SELECT
         c.city_name
FROM
         sql.shipment s
JOIN sql.city c ON s.city_id = c.city_id

EXCEPT

SELECT
         cc.city_name
FROM
         sql.driver d 
JOIN sql.city cc ON d.city_id=cc.city_id
ORDER BY 1

Задание 7.1
Выведите список zip-кодов, которые есть в таблице sql.driver, но отсутствуют в таблице sql.customer. Отсортируйте по возрастанию, столбец к выводу — zip.

SELECT
         d.zip_code zip
FROM
         sql.driver d
JOIN sql.shipment s ON s.driver_id = d.driver_id
EXCEPT
SELECT
         c.zip
FROM
         sql.customer c 
JOIN sql.shipment s ON s.cust_id = c.cust_id
ORDER BY 1

## 8. INTERSECT

вывести совпадающие по названию города и штаты.

SELECT 
         c.city_name object_name
FROM 
         sql.city c
INTERSECT
SELECT 
         cc.state
FROM 
         sql.city cc
ORDER BY 1

Как EXCEPT, так и INTERSECT убирают дубликаты, если они имеются.

Задание 8.1
Напишите запрос, который выведет список id городов, в которых есть и клиенты, и доставки, и водители.

SELECT 
         c.city_id
FROM 
         sql.city c
INTERSECT
SELECT 
         d.city_id
FROM 
         sql.driver d      
INTERSECT
SELECT
    cc.city_id
FROM
    sql.customer cc   
INTERSECT
SELECT
    s.city_id
FROM
    sql.shipment s

Задание 8.2
Выведите zip-код, который есть как в таблице с клиентами, так и в таблице с водителями.

SELECT 
         d.zip_code
FROM 
         sql.driver d
    join sql.city c on c.city_id = d.city_id
INTERSECT
SELECT 
         cc.zip
FROM 
         sql.customer cc
    join sql.city c on c.city_id = cc.city_id

## 9. Итоги. Закрепление знаний

Структура:

SELECT 
         N columns
FROM 
         table_1

UNION / UNION ALL / EXCEPT / INTERSECT 

SELECT 
         N columns
FROM 
         table_2

Задание 9.1
Выведите города с максимальным и минимальным весом единичной доставки. Столбцы к выводу — city_name, weight.

(
	SELECT
	   c.city_name,
	   s.weight
	FROM sql.shipment s
	   JOIN sql.city c ON s.city_id=c.city_id
	ORDER BY 2 desc
	LIMIT 1
)
UNION ALL
(
	SELECT
	   c.city_name,
	   s.weight
	FROM sql.shipment s
	   JOIN sql.city c ON s.city_id=c.city_id
	ORDER BY 2
	LIMIT 1
)


Задание 9.2
Выведите идентификационные номера клиентов (cust_id), которые совпадают с идентификационными номерами доставок (ship_id).
Столбец к выводу — mutual_id. Отсортируйте по возрастанию.

SELECT 
        c.cust_id mutual_id
FROM 
         sql.customer c

intersect

SELECT 
        s.ship_id
FROM 
         sql.shipment s
order by 1

Задание 9.3
Создайте справочник, содержащий уникальные имена клиентов, которые являются производителями (cust_type='manufacturer'), и производителей грузовиков, а также описание объекта — 'КЛИЕНТ' или 'ГРУЗОВИК'. Столбцы к выводу — object_name, object_description. Отсортируйте по названию в алфавитном порядке.

SELECT 
        c.cust_name object_name,
        'КЛИЕНТ' object_description
FROM 
         sql.customer c
where c.cust_type='manufacturer'

union

SELECT 
        t.make object_name,
        'ГРУЗОВИК' object_description
FROM 
         sql.truck t
         
order by 1